In [154]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
import numpy as np
import torch

# seq: joy, trust, fear, surprise, sadness, disgust, anger, anticipation
def emoMatchMetric(expect, actual):
    # model = AutoModelForSequenceClassification.from_pretrained("./firstSentenceDetection/") # local
    model = AutoModelForSequenceClassification.from_pretrained("Yuetian/roberta-large-finetuned-plutchik-emotion")
    # model = AutoModelForSequenceClassification.from_pretrained("Yuetian/bert-base-uncased-finetuned-plutchik-emotion")
    # assert model.config.output_attentions == True
    tokenizer = AutoTokenizer.from_pretrained("Yuetian/roberta-large-finetuned-plutchik-emotion")
    # tokenizer = AutoTokenizer.from_pretrained("Yuetian/bert-base-uncased-finetuned-plutchik-emotion")
    encoding = tokenizer(expect, return_tensors="pt")
    # tokenizer.decode(encoding['input_ids'])
    encoding = {k: v.to('cpu') for k,v in encoding.items()}
    outputs = model(**encoding)   
    # apply sigmoid + threshold
    logits = outputs.logits
    sigmoid = torch.nn.Sigmoid()
    probsExpect = sigmoid(logits.squeeze().cpu())
    
    encoding = tokenizer(actual, return_tensors="pt")
    # tokenizer.decode(encoding['input_ids'])
    encoding = {k: v.to('cpu') for k,v in encoding.items()}
    outputs = model(**encoding)   
    # apply sigmoid + threshold
    logits = outputs.logits
    sigmoid = torch.nn.Sigmoid()
    probsActual = sigmoid(logits.squeeze().cpu())
    
    probsExpect = probsExpect.detach().numpy()
    probsActual = probsActual.detach().numpy()
    # return [probsExpect, probsActual]
    return ((1 -probsExpect + probsActual)**2/1)

In [155]:
emoMatchMetric("I am happy", "I am glad")

array([1.0000405 , 1.0129129 , 1.0001721 , 3.9736207 , 1.000056  ,
       1.0004365 , 0.99998343, 0.07874388], dtype=float32)

2.8284271247461903

In [23]:
np.array([1, 1, 1, 1, 1, 1, 1, 1])


array([1, 1, 1, 1, 1])